In [2]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [5]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Welcome, and nice to meet you. I’m here to help with all kinds of things—explanations, writing and editing, brainstorming, learning new topics, solving problems, coding, and more.\n\nWhat would you like to do first? A few quick ideas:\n- Explain a concept in simple terms\n- Draft or edit an email, message, or essay\n- Plan a tiny project or trip\n- Write a short story or poem\n- Solve a math problem or code something\n- Learn a new topic or pick up a programming concept\n\nTell me your interests or just ask me anything, and we’ll go from there.'

In [6]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [8]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [10]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [11]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n.

## Time to bring it together - the API for OpenAI is very simple!

In [13]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [14]:
summarize("https://edwarddonner.com")

'# Edward Donner’s Corner of the Internet\n\nWelcome to Ed’s playground where code meets AI shenanigans. Ed’s a self-proclaimed DJ who\'s *very* amateur at electronic music but pretty pro at wrangling Large Language Models (LLMs). If you’re into AI battles, check out "Outsmart," where LLMs duke it out in diplomacy and deviousness — basically AI Game of Thrones.\n\nEd\'s not just playing around; he’s co-founder and CTO of Nebula.io, using AI to revolutionize talent discovery (because robots hiring humans is progress, right?). His prior AI startup, untapt, was acquired in 2021, so yeah, this guy’s got some chops. His company even patented a matching model and snagged some press – so it’s legit.\n\n### Latest AI-nerd News & Announcements:\n- **Nov 11, 2025:** The Unique Energy of an AI Live Event (because watching machines talk is apparently an “energy”)\n- **Sep 15, 2025:** AI in Production on AWS at Scale (glamorous day job stuff)\n- **May 28, 2025:** Be an AI Engineer and Leader: The C

In [15]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://edwarddonner.com")

# Edward Donner's Corner of the Web

Ed’s a code tinkerer and LLM experimenter who also DJs terribly and half-understands Hacker News. He’s the CTO/co-founder of Nebula.io, where AI meets talent scouting so recruiters can stop guessing and start hiring like pros. Oh, and he sold an AI startup in 2021, patented some fancy matching tech, and has customers and press fawning over his AI platform.

**Newsworthy Nuggets:**
- Nov 2025: Chatting up the "Unique Energy of an AI Live Event" (yes, in the future)
- Sept 2025: Dropping knowledge on Gen AI and Agentic AI scaling at AWS
- May 2025: AI engineer leadership courses and a 2025 exec AI briefing

Also, for your inner geek, there’s a “Connect Four” and “Outsmart” arena where LLMs duke it out in battles of wit and treachery. Follow, subscribe, or just admire from afar.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [17]:
display_summary("https://cnn.com")

# CNN Website: The Never-Ending News Buffet

Welcome to CNN, where you can drown in a tsunami of breaking news, politics, business, sports, style, celebrity gossip, climate doom, and even niche stuff like "Fear & Greed" (yes, a whole section for feelings). Also, buckle up for live TV, podcasts, and videos — if the ads ever let you watch them. 

The site is all about keeping you informed (and slightly overwhelmed) with sections for global chaos like the Ukraine-Russia and Israel-Hamas wars. Oh, and if any pesky ad glitches ruin your experience, they *really* want your feedback, probably to fine-tune those pesky auto-play ads crashing your zen.

In short, CNN is your one-stop shop for everything newsy, from Trump drama to the latest on the "2025 Elections," plus tips on mindfulness and luxury fashion. So whether you want to cry over the news or get inspired by "Innovative Cities," CNN’s got you covered — just prepare for some ad-induced patience training.

In [18]:
display_summary("https://anthropic.com")

# Anthropic: AI with a Conscience (and a Code Editor)

Anthropic is all about making AI that doesn’t go rogue and actually helps humanity instead of taking over the world. They’re a public benefit corporation—fancy talk for “we want AI to be nice, not naughty.” 

Their star product? Claude Opus 4.5, which is apparently the "best model in the world" for coding, multitasking like an overachieving intern, and handling enterprise workflows. If you didn’t know, Claude is their AI brainchild, with developer platforms, apps, and enough models named after poetic nonsense (Opus, Sonnet, Haiku) to make a bard jealous.

Newsflash: They just rolled out Claude Opus 4.5 with advanced tool use, which sounds like AI got a shiny new Swiss Army knife.

Overall, Anthropic wants to boldly go where no sketchy AI has gone before—balancing cutting-edge tech with some serious safety goggles on. Because if AI’s going to take over, they’d prefer it did so responsibly.